Justin Dano <br>
FE550 - Data Visualization Applications<br>
Assignment #3<br>
Due 10/11/2017<br>

#  Cryptocurrency Arbitrage - Liquidity and the Order Book

## 1. Introduction

As mentioned in previous assignments, the goal of the topic is to determine if trading cryptocurrencies can be a profitable endeavor. The previous two workbooks took an embryonic look at arbitrage opportunities present in the cryptocurrency markets. The research findings thus far has shown that arbitrage opportunities may indeed exist, and that they appear to be somewhat common. With that said, an investment strategy based on arbitrage trading may be feasible. The following analysis will dive deeper in cryptocurrency market microstructure and examine how much liquidity is actually available during an arbitrage opportunity. Specifically, the analysis aims to answer the following questions:   


<b> 1. How much liquidity do cryptocurrency exchanges have at a given moment in time? </b><br>
<b> 2. How much Bitcoin is available for trading during an arbitrage opportunity? </b><br>
<b> 3. Are cryptocurrency markets balanced?  </b>

The analysis begins with setting up a mechanism to pull order book data from different exchanges. This data will be  stored in a .csv file so the findings presented can be duplicated. Next, Bokeh will be used to visualize the order book for each exchange. To visualize the arbitrage opportunity, two order books from different exchanges will be compared simultaneously. With both order books lined up together, the arbitrage opportunity can be visualized, along with the liquidity available for the opportunity.     

#### Technology Stack
Python 3.6.1 <br>
Anaconda 3-4.4.0 <br>
Pandas 0.20.3 <br>
Bokeh 0.12.9 <br>
Developed on a Jupyter notebook.

In [1]:
# Import Modules and settings for styling
import urllib.request
import json
import time
import csv
import os
import numpy as np
import warnings
import pandas as pd
import pprint as pp
import locale
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearAxis, Range1d, ColumnDataSource, NumeralTickFormatter

locale.setlocale( locale.LC_ALL, '' )
warnings.filterwarnings('ignore')

chart_styling = {'axis_size':'12pt', 
                 'title_size':'14pt', 
                 'font':'times',
                 'legend_pos': 'top_right',
                 'legend_font': '8pt'}

output_notebook()

Loading BokehJS ...

## 2. Data Retrieval from CryptoWatch API

The data used in the previous workbooks were rudimentary at best. Price levels were calculated based on actual trades, and did not consider the various orders present in the order book. To have a more accurate analysis of arbitrage opportunities, the liquidity, or actual amount of volume that can be bought and sold must be known. 

Order book data will be retrieved from the [Crypto Watch](https://cryptowat.ch/) API. Since no freely available historic order book data could be found, the following functions work to pull the current order book. Next, the order book data is saved into a .csv file, which will called in the following section. Future work will involve setting up a websocket to pull data and store it in a database.

#### Note: The following section does not need to be ran for analysis. It is presented simply to demonstrate how the data was retrieved.

In [2]:
def write_to_csv(filename, data, directory):
    """
    Creates CSV file from order book data
    :param filename: name of csv file to be created
    :param data: list of prices/amounts
    """

    # Creates CSV file
    with open('order_book_data/' + str(directory) + '/' + filename, 'w') as csvfile:
        csvout = csv.writer(csvfile)
        for row in data:
            csvout.writerow(row)

            
def format_order_book_data_and_save(exchange, timestamp):
    """
    Formats order book data and saves into csv file
    :param exchange: String - name of exchange used in cryptowatch
    :param timestamp: String - timestamp of order book. used to create unique directory 
    """
    
    # Assemble order book for CSV. The bids and asks are saved in different files.
    order_book = json.load(exchange[0])['result']
    asks = order_book['asks']
    bids = order_book['bids']

    csv_asks_file = exchange[1] + '_asks.csv'
    csv_bids_file = exchange[1] + '_bids.csv'

    write_to_csv(csv_asks_file, asks, timestamp)
    write_to_csv(csv_bids_file, bids, timestamp)

            
def get_order_book_data(list_of_exchanges):
    """
    Make HTTP Requests to pull order book data from Cryptowatch
    :param list_of_exchanges: list of different exchanges
    :return:
        exchange_responses: list of tuples (HTTP Response for Order book, name of exchange)
        timestamp: string of timestamp of data retrieval
    """
    
    exchange_responses = []

    for exchange in list_of_exchanges:
        # Read in data from Cryptowatch
        url = 'https://api.cryptowat.ch/markets/' + exchange + '/btcusd/orderbook'
        exchange_responses.append((urllib.request.urlopen(url), exchange))

    # Time will be off by a few microseconds. This analysis will assume they are equivalent for each exchange.
    timestamp = pd.to_datetime(time.time(), unit='s')

    return exchange_responses, timestamp    


order_book_responses, eob_timestamp = get_order_book_data(['gdax', 'kraken', 'bitstamp'])

# Creates unique Directory
os.makedirs('order_book_data/' + str(eob_timestamp))

for response in order_book_responses:
    format_order_book_data_and_save(response, eob_timestamp)

## 3. Data Retrieval from CSV Files

The snapshot of the order book for each exchange has been created and saved in a timestamp named directory. Going forward, all of the analysis is done over one particular microsecond of the order book on October 10th, 2017 at 4:29:21.338556 PM. To test locally, please perform the following steps:

1. Comment out the current <i>directory_path</i> variable </b>
2. Uncomment out the second <i>directory_path</i> variable, and add the path to the directory containing the .csv data which was submitted.

The directory will be submitted with the exact timestamp presented below. Only thing that needs to be changed is the path where that directory is saved on the local computer.

In [3]:
def read_data(exchange, time_extension):
    """
    Reads order book data saved in .csv files
    :param exchange: String - identifies the order books exchange
    :param time_extension: String - timestamp of order book. Used to create unique directory
    :return: Dataframe - Order book data
    """    

    directory_path = r'/home/justin/PycharmProjects/bitcoin_arbitrage/order_book_data/' + time_extension
    #directory_path = r'your_path_to_submitted_data' + time_extension
    
    asks_file = directory_path + '/' + exchange + '_asks.csv'
    bids_file = directory_path + '/' + exchange + '_bids.csv'
  
    df_asks = pd.read_csv(asks_file, names=['ask_price', 'ask_volume'])
    df_bids = pd.read_csv(bids_file, names=['bid_price', 'bid_volume'])
    
    return pd.concat([df_bids, df_asks], axis=1)

sample_time = '2017-10-02 16:29:21.338665'
gdax_orderbook = read_data('gdax', sample_time)
kraken_orderbook = read_data('kraken', sample_time)
bitstamp_orderbook = read_data('bitstamp', sample_time)

gdax_orderbook.head()

,bid_price,bid_volume,ask_price,ask_volume
0,4408.43,6.1100,4408.44,1.330000
1,4408.42,2.0000,4408.45,0.010000
2,4408.35,0.2565,4408.46,0.010000
3,4408.20,0.2500,4408.50,0.098697
4,4408.05,0.0220,4409.08,3.670000


## 4. Calculating Liquidity 

To visualize the order book, the amount of liquidity at each order needs to be calculated. This is done by taking the cumulative size of each order.

In [4]:
def calculate_liquidity(order_book):
    """
    Calculates liquidity by taking cumulative sum of volume for bids and asks
    :param order_book: Dataframe - Order book with bids/asks
    :return: Dataframe - Order book with bids/asks and liquidity
    """
    order_book['ask_liquidity'] = order_book['ask_volume'].cumsum()
    order_book['bid_liquidity'] = order_book['bid_volume'].cumsum()

    # Reorder columns to align order book
    cols = ['bid_liquidity', 'bid_volume', 'bid_price', 'ask_price', 'ask_volume', 'ask_liquidity']

    return order_book[cols]

gdax_orderbook = calculate_liquidity(gdax_orderbook)
kraken_orderbook = calculate_liquidity(kraken_orderbook)
bitstamp_orderbook = calculate_liquidity(bitstamp_orderbook)

gdax_orderbook.head()

,bid_liquidity,bid_volume,bid_price,ask_price,ask_volume,ask_liquidity
0,6.1100,6.1100,4408.43,4408.44,1.330000,1.330000
1,8.1100,2.0000,4408.42,4408.45,0.010000,1.340000
2,8.3665,0.2565,4408.35,4408.46,0.010000,1.350000
3,8.6165,0.2500,4408.20,4408.50,0.098697,1.448697
4,8.6385,0.0220,4408.05,4409.08,3.670000,5.118697


## 5. Visualizing the Order Book

Visualizing the order book will be done by making use of Bokeh's <i>patches</i> glyph. Each side of the order book will be represented by its own patch. First, the coordinates that outline the shape of the orders will be created. This is accomplished in the <i>generate_plot_coordinates()</i> function. 

In [5]:
def generate_plot_coordinates(order_book, samples):
    """
    Generates the x and y coordinates used to create the shape of the order book.
    :param order_book: Dataframe - order book data to be visualized
    :param samples: Int - number of bids/asks to be plotted
    :return: Tuple - coordinates for the ask shape and bid shape
    """
    x_bid_coordinates = []
    y_bid_coordinates = []

    x_ask_coordinates = []
    y_ask_coordinates = []

    for i in range(samples):
        x_bid_coordinates.append(order_book['bid_price'][i])
        y_bid_coordinates.append(order_book['bid_liquidity'][i])

        x_ask_coordinates.append(order_book['ask_price'][i])
        y_ask_coordinates.append(order_book['ask_liquidity'][i])

    # Need to repeat first and last coordinate to create the bottom part of shape
    x_bid_coordinates = [x_bid_coordinates[0]] + x_bid_coordinates + [x_bid_coordinates[-1]]
    y_bid_coordinates = [0] + y_bid_coordinates + [0]

    x_ask_coordinates = [x_ask_coordinates[0]] + x_ask_coordinates + [x_ask_coordinates[-1]]
    y_ask_coordinates = [0] + y_ask_coordinates + [0]

    return x_bid_coordinates, x_ask_coordinates, y_bid_coordinates, y_ask_coordinates 

The next function <i>plot_order_book()</i> is now used to create the plot and add styling to the visualization. It calls the previous function <i>generate_plot_coordinates()</i>. 

In [6]:
def plot_order_book(order_book, order_count, exchange, ob_time, styling):
    """
    Creates a visualization of the Order Book Bids/Asks
    :param order_book: Dataframe - Order Book data 
    :param exchange: String - Name of the order books exchange
    :param ob_time: String - Timestamp of order book
    :param styling: Dict - Styling parameters
    """
    
    # Get x/y coordinates for both bids and asks of order book
    order_book_coords  = generate_plot_coordinates(order_book, order_count)
           
    # Create a new plot with a title and axis labels
    plot_title = ' Order Book for ' + str(exchange) + ' at ' + ob_time
    p = figure(title=plot_title,
               x_axis_label='USD/BTC', 
               y_axis_label='Liquidity (BTC\'s)',
               plot_width=950, 
               plot_height=500
              )

    # Create source for the data points, colors, and legend
    source = ColumnDataSource(dict(
        x_axis = [order_book_coords[0], order_book_coords[1]],
        y_axis = [order_book_coords[2], order_book_coords[3]],
        color = ['green', 'red'],
        label = ['Bids', 'Asks']
    ))    
    
    # Plot order book with coordinates
    p.patches(xs='x_axis', ys='y_axis', color='color', legend='label', alpha=0.5, line_width=2, source=source)

    # Graph Formatting
    p.xaxis.formatter = NumeralTickFormatter(format="=$ 0,0[.]00")
    p.title.text_font_size = styling['title_size']
    p.xaxis.axis_label_text_font_size = styling['axis_size']
    p.yaxis.axis_label_text_font_size = styling['axis_size']
    
    p.title.text_font = styling['font']
    p.xaxis.axis_label_text_font = styling['font']
    p.yaxis.axis_label_text_font = styling['font']
   
    show(p)

In [7]:
# Parameter to define the top bids/asks to visualize
order_count = 99
plot_order_book(bitstamp_orderbook, order_count, 'Bitstamp', sample_time, chart_styling)   

In [8]:
plot_order_book(kraken_orderbook, order_count, 'Kraken', sample_time, chart_styling)             

In [9]:
plot_order_book(gdax_orderbook, order_count, 'GDAX', sample_time, chart_styling)     

Each order book seems to have unique characteristics. It is interesting that Bitstamp had a noticeable spread compared to the other exchanges. GDAX looks like it has the most balanced order book at this particular moment, while both Kraken and Bitstamp clearly have greater buy interest. Also, it is important to note that each visualization is defined by the parameter <i>order_count</i> listed above. The parameter is set for the top 99 orders from each exchange (max capacity from [Crypto Watch](https://cryptowat.ch/)). Unfortunately, it is hard to compare different exchanges like this, with the axes unaligned. The next section will combine different order books on the same visualization so comparison can be done easier.


## 6. Visualizing the Arbitrage

The next function, <i>plot_dual_order_books()</i> will be used to show two order books simultaneously. It was designed with a large number of parameters to provide flexibility when creating the visualization.

In [10]:
def plot_dual_order_books(ob1, ob2, order_count, e1, e2, ob_time, x_scale, y_scale, styling, align_flag):
    """
    Creates a visualization of two order books on the same graph
    :param ob1: Dataframe - Order book from first exchange
    :param ob2: Dataframe - Order book from first exchange
    :param order_count: Int - number of orders to show in order book (max=99)
    :param e1: String - Name of exchange for ob1_coords 
    :param e2: String - Name of exchange for ob2_coords
    :param ob_time: String - Timestamp
    :param x_scale: List of ints - parameter to define how the graph displays price
    :param y_scale: List of ints - parameter to define how the graph displays liquidity
    :param styling: Dict - Styling parameters
    :param align_flag: Bool - Determines whether vertical lines should be added or not
    """
    
    # Get x/y coordinates for both order books, including bids and asks up to {order_count} 
    ob1_coords  = generate_plot_coordinates(ob1, order_count)   
    ob2_coords  = generate_plot_coordinates(ob2, order_count)    
    
    plot_title = ' Order Book for ' + str(e1) + ' and ' + str(e2) + ' at ' + ob_time
          
    # Create a new plot with a title and two axes
    p = figure(title=plot_title, 
               x_axis_label='USD/BTC', 
               y_axis_label= str(e1) + ' Liquidity (BTC\'s)',
               plot_width=950, 
               plot_height=500,
               x_range=Range1d(x_scale[0], x_scale[1]),
               y_range=Range1d(y_scale[0], y_scale[1]),
               toolbar_location='above'
              )

    # Create second axis
    p.extra_y_ranges = {'ex2': Range1d(start=y_scale[1], end=y_scale[0])}
    
    # Adding the second axis to the plot.  
    p.add_layout(LinearAxis(y_range_name="ex2", axis_label= str(e2) + ' Liquidity (BTC\'s)'), 'right')    
        
    # Create source for first exchange, including coordinates, colors, and legend
    source1 = ColumnDataSource(dict(
        x_axis = [ob1_coords[0], ob1_coords[1]],
        y_axis = [ob1_coords[2], ob1_coords[3]],
        color = ['lightskyblue', 'mediumpurple'],
        label = [str(e1) + ' Bids', str(e1) + ' Asks']
    ))    
   
    # Create source for second exchange, including coordinates, colors, and legend
    source2 = ColumnDataSource(dict(
        x_axis = [ob2_coords[0], ob2_coords[1]],
        y_axis = [ob2_coords[2], ob2_coords[3]],
        color = ['green', 'red'],
        label = [str(e2) + ' Bids', str(e2) + ' Asks']
    )) 
       
    # Plot the first order book 
    p.patches(xs='x_axis', ys='y_axis', color='color', legend='label', 
              alpha=0.5, line_width=2, source=source1)

    # Plot the second order book
    p.patches(xs='x_axis', ys='y_axis', color='color', legend='label', 
              alpha=0.5, line_width=2, y_range_name="ex2", source=source2)
    
    # Adds vertical lines to align the arbitrage
    if align_flag:
        left_vertical = (ob1_coords[1][0], (y_scale[0], y_scale[1]))
        p.line(left_vertical[0], left_vertical[1])  
        right_vertical = (ob2_coords[0][0], (y_scale[0], y_scale[1]))
        p.line(right_vertical[0], right_vertical[1])        
    
    # Graph Formatting
    p.xaxis.formatter = NumeralTickFormatter(format="=$ 0,0[.]00")
    p.title.text_font_size = styling['title_size']
    p.xaxis.axis_label_text_font_size = styling['axis_size']
    p.yaxis.axis_label_text_font_size = styling['axis_size']   
    p.title.text_font = styling['font']
    p.xaxis.axis_label_text_font = styling['font']
    p.yaxis.axis_label_text_font = styling['font']
    p.legend.label_text_font = styling['font']
    p.legend.location = styling['legend_pos']
    p.legend.label_text_font_size = styling['legend_font']

    show(p)            

In [11]:
# Parameters
order_count = 99
x_scale = [4270, 4510]
y_scale = [0, 500]
chart_styling['legend_pos'] = 'bottom_right'

plot_dual_order_books(gdax_orderbook, kraken_orderbook, order_count, 
                      'GDAX', 'Kraken', sample_time, x_scale, y_scale, chart_styling, False)

The image above takes a look at the top 99 bids and asks for both GDAX and Kraken exchanges. At first glance, it might appear that Kraken is the larger exchange, however the truth is quote the opposite. GDAX has so many more orders, that its first 99 bids/asks are only slightly different in price. Now that the order books are aligned on the same price scale, is becomes easier to see where the prices cross. However, improvements can be made to the scaling to allow for a more readable view of the arbitrage opportunity. 

First, by enabling the <i>align_flag</i> to true, two vertical lines form a visual aid.

In [12]:
plot_dual_order_books(gdax_orderbook, kraken_orderbook, order_count, 
                      'GDAX', 'Kraken', sample_time, x_scale, y_scale, chart_styling, True)  

The graph is still somewhat hard to see, so the next step is changing the <i>order_count, x_scale, and y_scale</i> parameters. The next three graphs give a better view of the arbitrage, and show how much liquidity is available.

In [13]:
order_count = 99
x_scale = [4350, 4460]
y_scale = [0, 300]
chart_styling['legend_pos'] = 'bottom_left'

plot_dual_order_books(gdax_orderbook, kraken_orderbook, order_count, 
                      'GDAX', 'Kraken', sample_time, x_scale, y_scale, chart_styling, True)  

In [14]:
# Parameters
order_count = 50
x_scale = [4400, 4420]
y_scale = [0, 100]

chart_styling['legend_pos'] = 'bottom_right'

plot_dual_order_books(gdax_orderbook, kraken_orderbook, order_count, 
                      'GDAX', 'Kraken', sample_time, x_scale, y_scale, chart_styling, True)  

In [15]:
# Parameters
order_count = 25
x_scale = [4405, 4415]
y_scale = [0, 30]
chart_styling['legend_pos'] = 'bottom_right'

plot_dual_order_books(gdax_orderbook, kraken_orderbook, order_count, 
                      'GDAX', 'Kraken', sample_time, x_scale, y_scale, chart_styling, True)  

The final graph shows the top 25 orders, but is scaled on the x-axis ($/BTC) to zoom in on the arbitrage, with the y-axis (volume) scaled down to 30 BTC's. To get an intuition about how much Bitcoin could actually be purchased, its whichever shape (liquidity) is smaller between the two horizontal lines. In this case, its GDAX Asks which has the smaller shaper compared to Kraken Bids. So the arbitrage would include buying GDAX and selling Kraken. It can also be roughly estimated that about 5 Bitcoins can be traded.

## 7. Calculating Arbitrage Profit

This section will be dedicated to calculating the actual profit that could be achieved by this arbitrage. The logic presented below is based on the assumption that purchase of Bitcoin from GDAX and selling of Bitcoin from Kraken occur simultaneously. Of course this is not realistic, but the process acts as a prerequisite for calculating profits in an actual trading model. First, a review of the order books from both GDAX and Kraken.

In [16]:
gdax_orderbook.head(10)

,bid_liquidity,bid_volume,bid_price,ask_price,ask_volume,ask_liquidity
0,6.11000,6.11000,4408.43,4408.44,1.330000,1.330000
1,8.11000,2.00000,4408.42,4408.45,0.010000,1.340000
2,8.36650,0.25650,4408.35,4408.46,0.010000,1.350000
3,8.61650,0.25000,4408.20,4408.50,0.098697,1.448697
4,8.63850,0.02200,4408.05,4409.08,3.670000,5.118697
5,13.18239,4.54389,4408.01,4409.20,0.010000,5.128697
6,14.22039,1.03800,4408.00,4409.44,0.022700,5.151397
7,14.24339,0.02300,4407.00,4410.03,0.010000,5.161397
8,14.29339,0.05000,4406.91,4410.44,0.879000,6.040397
9,16.79339,2.50000,4405.60,4410.63,0.161014,6.201411


In [17]:
kraken_orderbook.head(10)

,bid_liquidity,bid_volume,bid_price,ask_price,ask_volume,ask_liquidity
0,3.399,3.399,4410.1,4410.5,0.014,0.014
1,10.198,6.799,4410.0,4410.6,0.224,0.238
2,10.282,0.084,4407.2,4412.3,0.151,0.389
3,10.302,0.020,4405.8,4414.7,1.500,1.889
4,10.314,0.012,4405.1,4416.1,0.428,2.317
5,11.271,0.957,4405.0,4418.8,0.005,2.322
6,12.271,1.000,4403.7,4421.3,1.372,3.694
7,13.771,1.500,4402.9,4421.4,0.290,3.984
8,13.781,0.010,4402.3,4422.5,0.180,4.164
9,14.357,0.576,4401.6,4422.6,2.938,7.102


The following method <i>determine_arbitrage_orders()</i> will determine the maximum liquidity that can be used for the trade (the smaller of the two shapes) and calculates which orders to be bought and then sold.

In [18]:
def determine_arbitrage_orders(ob1, ob2):
    """
    Filters the orders that can be bough and sold for an arbitrage
    :param ob1: Dataframe - The first order book 
    :param ob2: Dataframe - The second order book
    :return: Tuple of DataFrames - filtered by orders that con be arbitraged
    """
    
    # Determine which exchange is cheaper to buy
    low_exchange = pd.DataFrame()
    high_exchange = pd.DataFrame()
    
    if ob1['ask_price'][0] > ob2['ask_price'][0]:
        low_exchange = ob2[['ask_price', 'ask_volume', 'ask_liquidity']].copy()
        high_exchange = ob1[['bid_price', 'bid_volume', 'bid_liquidity']].copy()
    else:
        low_exchange = ob1[['ask_price', 'ask_volume', 'ask_liquidity']].copy()
        high_exchange = ob2[['bid_price', 'bid_volume', 'bid_liquidity']].copy()
      
    
    # Determine lowest ask from cheaper exchange
    lowest_ask = low_exchange['ask_price'][0]
    
    # Determine highest bid from other exchange
    highest_bid = high_exchange['bid_price'][0]
    
    
    # Get orders that can be arbitraged
    buy_orders = low_exchange[low_exchange['ask_price'] < highest_bid]   
    sell_orders = high_exchange[high_exchange['bid_price'] > lowest_ask]
    
    return buy_orders, sell_orders
    
       
buy_orders, sell_orders = determine_arbitrage_orders(gdax_orderbook, kraken_orderbook)

The following dataframes show that eight orders can be purchased from GDAX totaling 5.161397 BTC, while the next step would be to sell 5.161397 orders on Kraken, taking the first order, and a portion of the second order.

In [19]:
buy_orders

,ask_price,ask_volume,ask_liquidity
0,4408.44,1.330000,1.330000
1,4408.45,0.010000,1.340000
2,4408.46,0.010000,1.350000
3,4408.50,0.098697,1.448697
4,4409.08,3.670000,5.118697
5,4409.20,0.010000,5.128697
6,4409.44,0.022700,5.151397
7,4410.03,0.010000,5.161397


In [20]:
sell_orders

,bid_price,bid_volume,bid_liquidity
0,4410.1,3.399,3.399
1,4410.0,6.799,10.198


Finally, the method <i>calculate_profit()</i> will take the weighted sum of both orders to determine exactly how much profit would be made by this hypothetical arbitrage.

In [21]:
def calculate_profit(buy_orders, sell_orders):
    """
    Calculates and prints the total profit made from the arbitrage 
    :param buy_orders: DataFrame - orders to buy
    :param sell_orders: DataFrame - orders to sell
    """
    
    # Determine weighted price of buy orders
    buy_orders['total_price'] = (buy_orders['ask_price'] * buy_orders['ask_volume']).cumsum()

    # Get last sell order since it will most likely be a partial
    last_sell_order = sell_orders.iloc[-1]
    sell_orders = sell_orders[:-1]

    # Calculate the liquidity to be used for the final order
    remaining_liquidity = buy_orders.iloc[-1]['ask_liquidity']
    for index, order in sell_orders.iterrows():
        remaining_liquidity -= order['bid_liquidity']

    sell_orders['total_price'] = (sell_orders['bid_price'] * sell_orders['bid_volume']).cumsum()

    # Determine weighted price of the final partial trade
    last_partial_order_price = last_sell_order['bid_price'] * remaining_liquidity

    # Total selling orders with the partial
    total_sell_price = sell_orders.iloc[-1]['total_price'] + last_partial_order_price
    total_buy_price = buy_orders.iloc[-1]['total_price']
   
    print('Total price paid for buying: ' + locale.currency(total_buy_price, grouping=True))
    print('Total price received for selling: ' + locale.currency(total_sell_price, grouping=True))
    
    arbitrage_profit = total_sell_price - total_buy_price
    print('Arbitrage profit: ' + locale.currency(arbitrage_profit, grouping=True))
    
calculate_profit(buy_orders, sell_orders)

Total price paid for buying: $22,756.11
Total price received for selling: $22,762.10
Arbitrage profit: $5.99


## 8. Conclusion
The visualizations used in this lab has given some insight of the market microstructure of Bitcoin markets. The amount of liquidity could be gauged at by the order book, and the ability to determine just how balanced the book was could be estimated. In section 6, the comparison of two order books in the same visualization gave an idea of just how much liquidity is available during an arbitrage opportunity.

Keep in mind this was a very granulated analysis, focusing on just one microsecond of the order book. Future work will be done to look at the trends of the order book and how liquidity changes over time. Once the liquidity can be properly modeled, a more accurate understanding of how much profit can be made from an arbitrage trading strategy. 

References: <br>
    (1) http://rickyhan.com/jekyll/update/2017/09/24/visualizing-order-book.html
    